# Capella API: Search, Order, and View Tutorial

In [ ]:
# Required libraries:
# requests
# json
# folium
# rasterio
# from skimage import exposure
# from matplotlib import rcParams

In [ ]:
!pip install folium rasterio 

Your username and password must be saved in a .json file named 'credentials.json' and formatted as follows.

{"username": "yourusername","password": "xxxxxxxxx"}

### Import required libraries, build a print utility function, assign API endpoints and load Credentials

In [ ]:
import requests
import json

# JSON utility function
def p(data):
    print(json.dumps(data, indent=2))

# Capella API endpoints
URL = 'https://api.data.capellaspace.com'
token = '/token'
collections = '/catalog/collections'
catsearch = '/catalog/stac/search'
orders = '/orders/'

# Load username and password
with open('credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']

### Get and Print Access Token

In [ ]:
# Get token
r = requests.post(URL + token, 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))

accesstoken = r.json()["accessToken"]

# print the token
print("Access Token: " + accesstoken)

headers = {'Authorization':'Bearer ' + accesstoken}

### Print Available Collections

In [ ]:
# See what collections are available
r = requests.get(URL + collections, headers=headers)

# Print results
p(r.json())

### Post Search Filters, Print the Results

In [ ]:
# Post search filters
filters = {
  "bbox": [-121.1298,41.3422,-119.4747,42.1484], # lower left coodinate and upper right coordinate, in decimal degrees
  "time": "2019-01-01T00:00:00Z/2019-12-31T12:31:12Z", # start date/end date and time in YYYY-MM-DDTHH:MM:SSZ format
  "limit": 50, # overwrite the default pagination limit of 10, adjust as necessary
  "query": {"sar:instrument_mode": {"eq": "IW"}, # add query parameters to filter search results
            "sar:pass_direction": {"eq": "ascending"}, 
            "sar:type": {"eq":"GRD"}},
  "sort": [{"field": "dtr:start_datetime","direction": "desc"}],
}
headers = {'Content-Type': 'application/json',
  'Accept': 'application/geo+json', 'Authorization':'Bearer ' + accesstoken}
r = requests.post(URL + catsearch, json=filters, headers=headers)

# Inspect the results
p(r.json())

### View the Results of the Search

In [ ]:
# Display the results on a folium map
import folium
loc = [(filters["bbox"][1]+filters["bbox"][3])/2, (filters["bbox"][0]+filters["bbox"][2])/2] 

m = folium.Map(location=[32, -118],zoom_start=4)
folium.GeoJson(
    r.text,
).add_to(m)

m

### Make and Post an Order

In [ ]:
# Build the Order
features = r.json()["features"]
granulelist = []

# Loop over all the features from the response and add to an array for an order
for f in features:
    item = {"CollectionId": f["collection"], "GranuleId": f["id"]}
    granulelist.append(item)
    
myorder = {"Items": granulelist}

# Post the order and inspect the result
r = requests.post(URL + orders, json=myorder, headers=headers)
p(r.json())

### Get the STAC records with the signed URLs using the /download endpoint, Print the Result

In [ ]:
myorderid = r.json()["orderId"]
r = requests.get(URL + orders + myorderid + '/download', headers=headers)
p(r.json())

### Visualize the Results

A Cloud Optimized GeoTIFF (COG) is a geoTIFF file which is hosted on a cloud such as AWS S3, Microsoft Azure, or Apache. The COG's format is optimized for remote reads. This section reads the COG for the downloaded asset.

In [ ]:
from matplotlib import rcParams
import cv2
import numpy as np

#Open and inspect one of the bands
features = r.json()

filepath = features[0]["assets"]["VH"]["href"]
print(filepath)# print the url to download this asset

# Open the file with Rasterio
import rasterio
from skimage import exposure
from matplotlib import pyplot as plt
Session = rasterio.Env(
            GDAL_DISABLE_READDIR_ON_OPEN='YES',
            CPL_VSIL_CURL_USE_HEAD='NO',
            )

with Session:
    with rasterio.open(filepath) as src:
        oviews = src.overviews(1)

        # Retrieve the smallest thumbnail
        oview = oviews[-1]
        print('Decimation factor= {}'.format(oview))
        # NOTE this is using a 'decimated read' (http://rasterio.readthedocs.io/en/latest/topics/resampling.html)
        thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview))).astype(np.uint8)
        equ = exposure.adjust_log(thumbnail, gain=.5)
        rcParams['figure.figsize'] = 15,8
        plt.imshow(equ, 'gray')
        plt.title('Overview - Band 4 {}'.format(thumbnail.shape))
        plt.xlabel('Column #')
        plt.ylabel('Row #')
